# Sistema unificado dos processadores de texto

#### Imports

In [7]:
from itertools import pairwise
from bitarray import bitarray

#### BMH

In [8]:
class BMH:
  def __init__(self, texto: str, padrão: str):
    self.texto, self.padrão = texto, padrão
    self.tamt, self.tamp = len(texto), len(padrão)
    self.calculate(); self.execute()
    print(f"Dicionário de saltos: {self.salto}")
    print(f"Indices percorridos: {self.index}")
    print(f"Matchs encontrados: {self.match}")

  def calculate(self):
    tamp, padrão = self.tamp, self.padrão
    tuplas = zip(padrão, range(1, tamp)[::-1])
    salto = {char: num for char, num in tuplas}
    if padrão[-1] not in salto:
      salto[padrão[-1]] = tamp
    self.salto = salto

  def execute(self):
    index, match = [], []
    tam, atual = self.tamp, 0
    while atual <= (self.tamt - tam):
      index.append(atual)
      word = self.texto[atual:atual+tam]
      if self.padrão == word:
        match.append(atual)
      try: atual += self.salto[word[-1]]
      except KeyError: atual += tam
    self.index = index
    self.match = match

#### ShiftAnd

In [9]:
class ShiftAnd:
  def __init__(self, texto: str, padrão: str):
    self.T, self.P, self.L = texto, padrão, len(padrão)
    self.calculate(); self.execute()
    print(f"Tabela das máscaras: {self.masks}")
    print(f"Matchs encontrados: {self.match}")

  def calculate(self):
    init = bitarray("0" * self.L)
    masks = {char: init.copy() for char in set(self.P)}
    for i, char in enumerate(self.P): masks[char][i] = 1
    self.masks = masks

  def execute(self):
    def execute_core():
      R = C = bitarray("0" * self.L)
      OR = bitarray("1" + ("0" * (self.L - 1)))
      for i, char in enumerate(self.T):
        try: R = ((R >> 1) | OR) & self.masks[char]
        except KeyError: R = C
        if R[-1]: yield f"no índice {i}"
    self.match = ", ".join(execute_core())

#### ShiftAndAprox

In [10]:
class ShiftAndAprox:
  def __init__(self, texto: str, padrão: str, k = 0):
    self.T, self.P, self.L = texto, padrão, len(padrão)
    self.calculate(); self.execute(k)
    print(f"Tabela das máscaras: {self.masks}")
    print(f"Matchs encontrados: {self.match}")

  def calculate(self):
    init = bitarray("0" * self.L)
    masks = {char: init.copy() for char in set(self.P)}
    for i, char in enumerate(self.P): masks[char][i] = 1
    self.masks = masks

  def execute(self, k: int):
    def execute_core():
      RK = [bitarray("1" * i + "0" * (self.L - i)) for i in range(k+1)]
      CK = AT = [*RK]; OR = bitarray("1" + ("0" * (self.L - 1)))
      for i, char in enumerate(self.T):
        try: RK[0] = ((AT[0] >> 1) | OR) & self.masks[char]
        except KeyError: RK[0] = CK[0]
        if RK[0][-1]: yield f"exato em {i}"
        for a, n in pairwise(range(k+1)):
          try: escolha = self.masks[char]
          except KeyError: escolha = CK[n]
          RK[n] = ((AT[n] >> 1) & escolha) | AT[a] | (RK[a] >> 1) | OR
          if RK[n][-1]: yield f"aprox. em {i}"
        AT = [*RK]
    self.match = ", ".join(execute_core())

#### Sistema que unifica os códigos

In [11]:
def sistema(texto: str, padrão: str, k = 0):
  if k:
    print("ShiftAndAprox:")
    ShiftAndAprox(texto, padrão, k)
  else:
    print("BMH:")
    BMH(texto, padrão)
    print("\nShiftAnd:")
    ShiftAnd(texto, padrão)

#### Executando o mesmo

In [13]:
sistema("os testes testam os alunos com um teste de capacidade", "teste", 1)

ShiftAndAprox:
Tabela das máscaras: {'s': bitarray('00100'), 't': bitarray('10010'), 'e': bitarray('01001')}
Matchs encontrados: aprox. em 6, exato em 7, aprox. em 7, aprox. em 8, aprox. em 11, aprox. em 13, aprox. em 37, exato em 38, aprox. em 38, aprox. em 39
